In [1]:
import pysam
import numpy as np
import os
import math
from scipy import stats

In [10]:
ch = ['chr'+str(i) for i in range(1,23)]
vcf_file_path = '/Users/ksg/gene_files/hg002/vcf/只有PASS/'
depth_file_path = '/Volumes/big_ksg/hg002/ngmlr/chr_depth_file/'
save_chr_single_area_path = r'/Users/ksg/gene_files/hg002/chr_single_area_normal/'

In [2]:
def binarySearch (arr, l, r, x): 

    if r >= l: 
        mid = int(l + (r - l)/2)
        if int(arr[mid].strip().split('\t')[1]) == x: 
            return mid 
        elif int(arr[mid].strip().split('\t')[1]) > x: 
            return binarySearch(arr, l, mid-1, x) 
        else: 
            return binarySearch(arr, mid+1, r, x)   
    else: 
        return -1

In [3]:
def merge_del_area(vcf_file_path,chid): 
    open_file_1 = open(vcf_file_path+chid+".txt")
    open_vcf_file = open_file_1.readlines() #'chr18\t10050\t10659\t1\n'
    remp = []
    j = 0
    for i in range(len(open_vcf_file)-1):
        if i == j:
            if (int(open_vcf_file[i+1].strip().split('\t')[1]) - int(open_vcf_file[i].strip().split('\t')[2])) > 0:
                remp.append(int(open_vcf_file[i].strip().split('\t')[1]))
                remp.append(int(open_vcf_file[i].strip().split('\t')[2]))
                j += 1
            elif (int(open_vcf_file[i+1].strip().split('\t')[1]) - int(open_vcf_file[i].strip().split('\t')[2])) <= 0:
                remp.append(int(open_vcf_file[i].strip().split('\t')[1]))
                if open_vcf_file[i+1].strip().split('\t')[2] >= open_vcf_file[i].strip().split('\t')[2]:
                    remp.append(int(open_vcf_file[i+1].strip().split('\t')[2]))
                else:
                    remp.append(int(open_vcf_file[i].strip().split('\t')[2]))
                j += 2
    merged_del_area = np.array(remp).reshape((-1,2))
    print(chid+' merge_del_area ... done')
    open_file_1.close()
    return merged_del_area 

In [4]:
def zero_area(vcf_file_path,depth_file_path,chid): 
    zero_area = []
    del_zero_area_index = []
    merged_del_area = merge_del_area(vcf_file_path,chid) 
    merged_del_area_len = len(merged_del_area) 
    open_depth = open(depth_file_path+chid+"_depth.txt") 
    open_depth_file = open_depth.readlines()
    open_depth_file_len = len(open_depth_file) 
    for i in range(merged_del_area_len):
        l_del_point = merged_del_area[i][0]
        r_del_point = merged_del_area[i][1] 
        l_point_index = binarySearch(open_depth_file,0,open_depth_file_len - 1,l_del_point)
        r_point_index = binarySearch(open_depth_file,0,open_depth_file_len - 1,r_del_point)                                   
        if l_point_index != -1 and r_point_index != -1 and (l_point_index-r_point_index) == (l_del_point-r_del_point):
            zero_area.append(l_del_point)
            zero_area.append(l_point_index)
            zero_area.append(r_del_point)
            zero_area.append(r_point_index)
    zero_area.insert(0,binarySearch(open_depth_file,0,open_depth_file_len - 1,int(open_depth_file[0].strip().split('\t')[1])))
    zero_area.insert(0,int(open_depth_file[0].strip().split('\t')[1]))
    zero_area.append(int(open_depth_file[-1].strip().split('\t')[1]))
    zero_area.append(binarySearch(open_depth_file,0,open_depth_file_len - 1,int(open_depth_file[-1].strip().split('\t')[1])))
    #print(zero_area[:5],len(zero_area))
    zero_array = np.array(zero_area).reshape((-1,4))[:,[0,2]]
    for i in range(len(zero_array)):
        if (zero_array[i][1]-zero_array[i][0])<100000: 
            del_zero_area_index.append(i)
    f_zero_array = np.delete(zero_array,del_zero_area_index,axis=0)
    final_zero_array = []
    for i in range(len(f_zero_array)):
        l_zero_point = f_zero_array[i][0]
        r_zero_point = f_zero_array[i][1]
        l_zero_point_index = binarySearch(open_depth_file,0,open_depth_file_len - 1,l_zero_point)
        r_zero_point_index = binarySearch(open_depth_file,0,open_depth_file_len - 1,r_zero_point)
        if (l_zero_point_index-r_zero_point_index) == (l_zero_point-r_zero_point):
            final_zero_array.append(f_zero_array[i])
    open_depth.close()
    print(chid+' zero_area ... done')
    return np.array(final_zero_array),open_depth_file 


def get_zero_feature_num(vcf_file_path,depth_file_path,chid,get_num=10000): 

    final_zero_array,open_depth_file = zero_area(vcf_file_path,depth_file_path,chid)
    final_zero_array_length = final_zero_array[:,1] - final_zero_array[:,0]
    final_zero_array_length_sum = np.sum(final_zero_array,axis=0)[1] - np.sum(final_zero_array,axis=0)[0]
    every_zero_area_num = (final_zero_array_length / final_zero_array_length_sum * get_num)+1
    print(chid+' get_zero_sample_num ... done')
    return final_zero_array,every_zero_area_num.astype(int),open_depth_file,len(open_depth_file)

def get_final_zero_area(final_zero_array,every_zero_area_num,reserved_length=1000,window_length=100): 
    random = np.random.RandomState(4396)
    final_zero_areas = []
    for i in range(len(final_zero_array)):
        single_area_zero_num = random.randint(low=final_zero_array[i][0]+reserved_length,high=final_zero_array[i][1]-reserved_length, size=every_zero_area_num[i])
        for j in range(len(single_area_zero_num)):
            final_zero_areas.append([single_area_zero_num[j],single_area_zero_num[j]-1+window_length])
    return final_zero_areas

In [11]:
for chid in ch:
    a,b,c,d = get_zero_feature_num(vcf_file_path,depth_file_path,chid,get_num=10000)
    e = get_final_zero_area(a,b)
    save_file = open(save_chr_single_area_path+chid+'_single_normal_area.txt',"w+")
    for i in e:
        save_file.write(str(i[0])+"\t"+str(i[1])+"\n")
    save_file.close()

chr1 merge_del_area ... done
chr1 zero_area ... done
chr1 get_zero_sample_num ... done
chr2 merge_del_area ... done
chr2 zero_area ... done
chr2 get_zero_sample_num ... done
chr3 merge_del_area ... done
chr3 zero_area ... done
chr3 get_zero_sample_num ... done
chr4 merge_del_area ... done
chr4 zero_area ... done
chr4 get_zero_sample_num ... done
chr5 merge_del_area ... done
chr5 zero_area ... done
chr5 get_zero_sample_num ... done
chr6 merge_del_area ... done
chr6 zero_area ... done
chr6 get_zero_sample_num ... done
chr7 merge_del_area ... done
chr7 zero_area ... done
chr7 get_zero_sample_num ... done
chr8 merge_del_area ... done
chr8 zero_area ... done
chr8 get_zero_sample_num ... done
chr9 merge_del_area ... done
chr9 zero_area ... done
chr9 get_zero_sample_num ... done
chr10 merge_del_area ... done
chr10 zero_area ... done
chr10 get_zero_sample_num ... done
chr11 merge_del_area ... done
chr11 zero_area ... done
chr11 get_zero_sample_num ... done
chr12 merge_del_area ... done
chr12 z